In [1]:
# импорт необходимых библиотек
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup
import aiohttp
import asyncio

from typing import *

import re

from pathlib import Path

import warnings
warnings.filterwarnings('ignore')
# from datetime import datetime

# Parsing

## Парсинг категорий

In [15]:
def parse_categories(url: str, no_cat: int) -> List[List[str]]:
    """
    Парсит категории с указанного URL-адреса.

    Parameters
    ----------
    url : str
        URL-адрес для парсинга.
    no_cat : int
        Номер категории для парсинга.

    Returns
    -------
    List[List[str]]
        Список названий категорий и соответствующих URL-адресов.

    """
    all_categories = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    categories = soup.find('dd', {
        'data-catalog-content': f'{no_cat}'
    }).find_all('a')
    for category in categories:
        name = category.text
        url_category = 'https://papteki.ru' + category['href']
        all_categories.append([name, url_category])
    return all_categories


def parse_each_category(name_category: str, url_category: str) -> None:
    """
    Парсит категорию и записывает результаты в файл CSV.

    Parameters
    ----------
    name_category : str
        Название категории.
    url_category : str
        URL-адрес категории.
    Returns
    -------
    None
    """
    params = {'p': 1}
    pages = 1
    data_category = []

    # цикл по каждой странице категории
    while params['p'] <= pages:
        response = requests.get(url_category, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')
        items_category = soup.find_all('div',
                                       class_='col-xl-4 col-lg-4 col-md-6')
        # парсинг товаров со страницы
        if items_category:
            for item in items_category:
                name = item.find('a',
                                 class_='catalog__item-title').text.strip()
                price = item.find('div',
                                  class_='catalog__item-price').text.strip()
                producer = item.find(
                    'div',
                    class_='catalog__item-name').text.split(':')[1].strip()
                img_card = 'https://papteki.ru' + item.find('img')['src']
                url_card = 'https://papteki.ru' + item.find(
                    'a')['href'].strip()
                data_category.append(
                    [name, price, producer, img_card, url_card, name_category])
        else:
            break

        # установка номера последней страницы
        if len(soup.find_all('a', class_='page-link')) == 1:
            last_page_num = int(
                soup.find_all('a', class_='page-link')[-1].text)
        else:
            last_page_num = int(
                soup.find_all('a', class_='page-link')[-2].text)
        pages = last_page_num if pages < last_page_num else pages
        params['p'] += 1

    # запись в файл csv результата парсинга категории
    df_cards = pd.DataFrame(
        data_category,
        columns=['name', 'price', 'producer', 'img', 'url', 'category'])
    df_cards.to_csv(f'data/drug/categories/data_cards_{name_category}.csv',
                    index=0)

In [16]:
url = 'https://papteki.ru/'

# код для парсинга данных карточек товаров с сайта по категориям лекарств
lst_cat_drug = parse_categories(url=url, no_cat=128)
for name_category, url_category in lst_cat_drug:
    parse_each_category(name_category, url_category)

# код для парсинга данных карточек товаров с сайта по категориям БАД
lst_cat_supp = parse_categories(url=url, no_cat=16)
for name_category, url_category in lst_cat_supp:
    parse_each_category(name_category, url_category)
    


In [ ]:
# # код для парсинга данных карточек товаров по категориям косметики
# lst_cat_supp = parse_categories(url=url, no_cat=73)
# for name_category, url_category in lst_cat_supp:
#     parse_each_category(name_category, url_category)

# # код для парсинга данных карточек товаров по категориям гигиены
# lst_cat_supp = parse_categories(url=url, no_cat=42)
# for name_category, url_category in lst_cat_supp:
#     parse_each_category(name_category, url_category)

# # код для парсинга данных карточек товаров по категориям для мамы и малыша
# lst_cat_supp = parse_categories(url=url, no_cat=59)
# for name_category, url_category in lst_cat_supp:
#     parse_each_category(name_category, url_category)
    
# # код для парсинга данных карточек товаров по категориям гигиены
# lst_cat_supp = parse_categories(url=url, no_cat=42)
# for name_category, url_category in lst_cat_supp:
#     parse_each_category(name_category, url_category)
    
# # код для парсинга данных карточек товаров по категориям мед техники
# lst_cat_supp = parse_categories(url=url, no_cat=265)
# for name_category, url_category in lst_cat_supp:
#     parse_each_category(name_category, url_category)
    
# # код для парсинга данных карточек товаров по категориям мед изделий
# lst_cat_supp = parse_categories(url=url, no_cat=273)
# for name_category, url_category in lst_cat_supp:
#     parse_each_category(name_category, url_category)

## Парсинг страниц товаров

*Асинхронный код написан при сотрудничестве со специалистом Python backend developer*

In [17]:
class AsyncListIterator:
    """
    Асинхронный итератор для списка.

    Parameters
    ----------
    lst : List[Any]
        Список элементов.

    Attributes
    ----------
    lst : List[Any]
        Список элементов.
    queue : asyncio.Queue
        Очередь для асинхронных операций.
    index : int
        Индекс текущего элемента в списке.
    """

    def __init__(self, lst: List[Any]):
        self.lst = lst
        self.queue = asyncio.Queue()
        self.index = 0

    def __aiter__(self) -> AsyncIterator[Any]:
        """
        Возвращает асинхронный итератор.

        Returns
        -------
        AsyncIterator[Any]
            Асинхронный итератор.
        """
        return self

    async def __anext__(self) -> Any:
        """
        Возвращает следующий элемент списка.

        Returns
        -------
        Any
            Следующий элемент списка.

        Raises
        ------
        StopAsyncIteration
            Если достигнут конец списка.
        """
        if self.index < len(self.lst):
            item = self.lst[self.index]
            self.index += 1
            return item
        else:
            raise StopAsyncIteration

In [18]:
TCP_CONNECTIONS = 10
N_WORKERS = 20
BATCH_SIZE = 10


def parse(doc: str) -> List[Optional[str]]:
    """
    Парсит информацию из HTML-документа.

    Parameters
    ----------
    doc : str
        HTML-документ.

    Returns
    -------
    List[Optional[str]]
        Список с информацией о товаре.
    """
    soup = BeautifulSoup(doc, 'html.parser')

    names_dict = {
        'Название': None,
        'Порядок отпуска': None,
        'Тип': None,
        'Цена': None,
        'Действующее вещество': None,
        'Форма выпуска': None,
        'Производитель': None,
        'Страна производства': None,
        'Фармакологическая группа': None,
        'Состав': None,
        'Описание': None,
        'Фармакологическое действие': None,
        'Показания к применению': None,
        'Применение при беременности и кормлении грудью': None,
        'Способ применения и дозировка': None,
        'Противопоказания': None,
        'Побочные действия': None,
        'Взаимодействие': None,
        'Передозировка': None,
        'Особые указания': None,
        'Условия хранения': None,
        'Ссылка на страницу препарата на сайте http://grls.rosminzdrav.ru/':
        None
    }

    names_list = list(names_dict.keys())

    names_dict['Название'] = soup.find('h1').text.strip()

    names_dict['Порядок отпуска'] = soup.find(
        'div', class_='card-product__dop-title').text.strip()

    if soup.find_all('div', class_='card-product__dop-title')[1].text.strip(
    ) == 'БАД, не является лекарственным средством':
        names_dict['Тип'] = 'supp'

    try:
        names_dict['Цена'] = soup.find(
            'span', class_='card-product__info-price').text.strip()
    except Exception:
        names_dict['Цена'] = 'n/a'

    items_part_1 = soup.find_all('div',
                                 class_='card-product__info-descrip-item')
    for k in items_part_1:
        for name in names_list[4:8]:
            if name in k.text:
                names_dict[name] = k.text.split(':')[1].strip()

    items_part_2 = soup.find_all('div', class_='card')

    for j in items_part_2:
        head = j.find('div', class_='card-header').text.strip()
        if head in names_list[8:]:
            names_dict[head] = j.find(
                'div', class_='card-product__instruction-tx').text.strip()

    return list(names_dict.values())


async def my_function(my_urls: str,
                      session: aiohttp.ClientSession) -> List[Optional[str]]:
    """
    Асинхронная функция для выполнения запросов и парсинга данных.

    Parameters
    ----------
    my_urls : str
        URL-адрес.
    session : aiohttp.ClientSession
        Сессия для выполнения запросов.

    Returns
    -------
    List[Optional[str]]
        Результаты парсинга.
    """
    try:
        async with session.get(my_urls, timeout=70) as response:
            resp = await response.text()
            return parse(resp)
    except Exception as e:
        await asyncio.sleep(2)
        async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(
                limit=TCP_CONNECTIONS, force_close=True)) as session:
            return await my_function(my_urls, session)


async def worker(queue: asyncio.Queue, session: aiohttp.ClientSession,
                 results: List[List[Optional[str]]]) -> None:
    """
    Асинхронная функция для обработки URL-адресов в очереди.

    Parameters
    ----------
    queue : asyncio.Queue
        Очередь с URL-адресами.
    session : aiohttp.ClientSession
        Сессия для выполнения запросов.
    results : List[List[Optional[str]]]
        Список для сохранения результатов.

    Returns
    -------
    None
    """
    while True:
        url = await queue.get()
        results.append(await my_function(url, session))
        queue.task_done()


async def main(URLS: List[str]) -> List[List[Optional[str]]]:
    """
    Асинхронная функция для выполнения параллельных запросов.

    Parameters
    ----------
    URLS : List[str]
        Список URL-адресов.

    Returns
    -------
    List[List[Optional[str]]]
        Результаты парсинга.
    """
    results = []
    queue = asyncio.Queue(N_WORKERS)

    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(
            limit=TCP_CONNECTIONS, force_close=True)) as session:

        workers = [
            asyncio.create_task(worker(queue, session, results))
            for _ in range(N_WORKERS)
        ]
        async for URL in URLS:
            await queue.put(URL)
        await queue.join()

    for w in workers:
        w.cancel()

    return results

In [19]:
async def parse_category_items(files_direct: str) -> None:
    """
    Парсит информацию о продуктах для каждой категории.

    Parameters
    ----------
    files_direct : str
        Директория с файлами категорий.

    Returns
    -------
    None
    """
    data_dir = Path(files_direct)

    # проход по файлам каждой категории
    for category_file in data_dir.glob('*.csv'):
        df_category = pd.read_csv(category_file)
        name_category = df_category['category'][0]

        # создание асинхронного итератора
        lst_urls = list(df_category['url'])
        async_list_urls = AsyncListIterator(lst_urls)

        # парсинг страниц товаров
        results = await main(async_list_urls)

        # запись результата в файл csv
        df_results = pd.DataFrame(
            results,
            columns=[
                'name', 'prescription', 'type', 'price', 'active_drug', 'form',
                'producer', 'country', 'group', 'contains', 'description',
                'action', 'use_for', 'pregnancy', 'how_use', 'aware_of',
                'side_effects', 'interaction', 'overdose', 'precautions',
                'storage', 'url_grls'
            ])
        df_results['category'] = name_category
        df_results.to_csv(
            f'data/drug/info_categories/data_info_{name_category}.csv',
            index=0)

In [24]:
# start = datetime.now()
await parse_category_items(files_direct='data/drug/categories')
# print(f'Время выполнения:{datetime.now() - start}')

## Добавление информации

In [36]:
def safe_extract_text(element: BeautifulSoup) -> str:
    """
    Безопасно извлекает текст из элемента BeautifulSoup.
    
    Parameters
    ----------
    element: BeautifulSoup
        Элемент BeautifulSoup.
    
    Returns
    -------
    str
        Извлеченный текст или None, если возникла ошибка.
    """
    try:
        return element.text.strip()
    except Exception:
        return None


def safe_extract_value(element: BeautifulSoup) -> str:
    """
    Безопасно извлекает значение атрибута 'value' из элемента BeautifulSoup.
    
    Parameters
    ----------
    element: BeautifulSoup
        Элемент BeautifulSoup.
    
    Returns
    -------
    str
        Извлеченное значение атрибута 'value' или None, если возникла ошибка.
    """
    try:
        return element.get('value')
    except Exception:
        return None


def parse_grls(files_direct: str) -> None:
    """
    Парсит информацию из HTML-документа.

    Parameters
    ----------
    files_direct : str
        Директория с файлами категорий.

    Returns
    -------
    None
        Записывает в файл csv информацию с сайта.
    """
    data_dir = Path(files_direct)
    for category_file in data_dir.glob('*.csv'):
        df_category = pd.read_csv(category_file)
        name_category = df_category['category'][0]
        lst_new_cols = [
            'TN', 'INN', 'DF', 'dose', 'date', 'temp', 'PG', 'ATC', 'VEM'
        ]
        df_category = df_category.reindex(
            columns=df_category.columns.tolist() + lst_new_cols)

        for index, url in enumerate(df_category['url_grls']):
            if isinstance(url, str):
                df_category.loc[index, 'type'] = 'drug'
                if '://grls.rosminzdrav' in url:
                    response = requests.get(url)
                    soup = BeautifulSoup(response.text, 'html.parser')
                    try:
                        df_category.loc[index, 'TN'] = safe_extract_value(
                            soup.find('input',
                                      {'name': "ctl00$plate$TradeNmR"}))
                        df_category.loc[index, 'INN'] = safe_extract_text(
                            soup.find('textarea',
                                      {'name': "ctl00$plate$Innr"}))
                        df_category.loc[index, 'DF'] = safe_extract_text(
                            soup.find('tr', class_='hi_sys').find_all('td')[0])
                        df_category.loc[index, 'dose'] = safe_extract_text(
                            soup.find('tr', class_='hi_sys').find_all('td')[1])
                        df_category.loc[index, 'date'] = safe_extract_text(
                            soup.find('tr', class_='hi_sys').find_all('td')[2])
                        df_category.loc[index, 'temp'] = safe_extract_text(
                            soup.find('tr', class_='hi_sys').find_all('td')[3])
                        df_category.loc[index, 'PG'] = safe_extract_text(
                            soup.find('table', {
                                'id': 'ctl00_plate_grFTG'
                            }).find('td'))
                        df_category.loc[index, 'ATC'] = safe_extract_text(
                            soup.find('table', {
                                'id': 'ctl00_plate_grATC'
                            }).find('td'))
                        df_category.loc[index, 'VEM'] = safe_extract_value(
                            soup.find('input',
                                      {'name': "ctl00$plate$txtNecessary"}))
                    except Exception as e:
                        print(f"An error occurred: {e}")

        df_category.to_csv(f'data/drug/result/data_{name_category}.csv',
                           index=0)

In [37]:
 parse_grls(files_direct='data/drug/info_categories')

An error occurred: 'NoneType' object has no attribute 'find'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType' object has no attribute 'find_all'
An error occurred: 'NoneType'

## Объединение файлов csv

In [38]:
def concat_files(files_to_concat: str) -> None:
    """
    Объединяет файлы CSV в один файл.

    Parameters
    ----------
    files_to_concat : str
        Путь к файлам CSV для объединения.

    Returns
    -------
    None
    """
    data_dir = Path(files_to_concat)
    df = pd.concat([pd.read_csv(f) for f in data_dir.glob("*.csv")],
                   ignore_index=True)
    df.to_csv("data/drug.csv", index=False)

In [39]:
concat_files(files_to_concat='data/drug/result')

# Preprocessing data

Проведем предобработку признаков

**Атрибуты**:

- name - наименование лекарственного препарата (ЛП)
- prescription - порядок отпуска (по рецепту/без рецепта)
- type - БАД или лекарственный препарат (ЛП)
- price - цена
- active_drug - действущее вещество
- form - форма выпуска
- producer - производитель
- country - страна производства
- group - фармакологическая группа
- contains - состав
- description - описание
- action - фармакологическое действие
- use_for - показания к применению
- pregnancy - применение при беременности и кормлении грудью
- how_use - способ применения и дозировка
- aware_of - противопоказания
- side_effects - побочные эффекты
- interaction - взаимодействие
- overdose - передозировка
- precautions - особые указания
- storage - условия хранения
- url_grls - ссылка на страницу препарата на сайте http://grls.rosminzdrav.ru/
- category - категория на сайте
- TN - торговое наименование (ТН) 
- INN - международное непатентованное наименование (МНН) 
- DF - лекарственная форма 
- dose - дозировка 
- date - срок годности 
- temp - условия хранения 
- PG - фармако-терапевтическая группа (ФТГ) 
- ATC - анатомо-терапевтическая химическая классификация (АТХ) 
- VEM - входит ли препарат в список ЖНВЛП

## Import data

In [2]:
# загрузим данные
df_drug = pd.read_csv('data/drug.csv')
df_supp = pd.read_csv('data/supplement.csv')

In [3]:
# объединим данные по БАД и ЛП в один датасет
df = pd.concat([df_drug, df_supp], ignore_index=True)

# удалим признак категории
df = df.drop('category', axis=1)

In [4]:
# посмотрим, сколько дупликатов
len(df)- len(df.drop_duplicates())

764

In [5]:
# удалим дупликаты
df = df.drop_duplicates()

In [6]:
# посмотрим на датасет
df.shape

(7877, 31)

In [7]:
df.head()

,name,prescription,type,price,active_drug,form,producer,country,group,contains,...,url_grls,TN,INN,DF,dose,date,temp,PG,ATC,VEM
0,Никсар тб 20 мг №30,Отпускается по рецепту,drug,1 700.50,Биластин,тб,А.Менарини Мэнюфекчеринг Лоджистикс энд Сервис...,NaN,противоаллергическое средство (H1-гистаминовых...,"Действующее вещество: биластин - 20,00 мг;\nВс...",...,http://grls.rosminzdrav.ru/Grls_View_v2.aspx?r...,Никсар®,Биластин,таблетки,20 мг,5 лет,При температуре не выше 30 град.,противоаллергическое средство - H1-гистаминовы...,R06AX29,Нет
1,Фенисмарт капли внут 1мг/мл 12мл,Отпускается без рецепта,drug,388.00,Диметинден,капли внут,Гленмарк Фармасьютикалз Лтд (Индия),NaN,противоаллергическое средство (H1-гистаминовых...,Действующим веществом является диметинден.\n1 ...,...,https://grls.rosminzdrav.ru/Grls_View_v2.aspx?...,Фенисмарт,Диметинден,капли для приема внутрь,1 мг/мл,2 года,"При температуре не выше 25 град., в оригинальн...",антигистаминные средства системного действия; ...,R06AB03,Нет
2,Димедрол р-р для в/в в/м введ 10мг/мл 5мл №10,Отпускается по рецепту,drug,от 87.00,Дифенгидрамин,р-р для в/в в/м введ,Новосибхимфарм (Россия),NaN,противоаллергическое средство (H1-гистаминовых...,В 1 мл содержится:\nактивное вещество: дифенги...,...,https://grls.rosminzdrav.ru/Grls_View_v2.aspx?...,Димедрол,Дифенгидрамин,раствор для внутривенного и внутримышечного вв...,10 мг/мл,5 лет,"В защищенном от света месте, при температуре 5...",противоаллергическое средство - H1-гистаминовы...,R06AA02,Да
3,Никсар тб 20 мг №10,Отпускается по рецепту,drug,735.50,Биластин,тб,А.Менарини Мэнюфекчеринг Лоджистикс энд Сервис...,NaN,противоаллергическое средство (H1-гистаминовых...,"Действующее вещество: биластин - 20,00 мг;\nВс...",...,http://grls.rosminzdrav.ru/Grls_View_v2.aspx?r...,Никсар®,Биластин,таблетки,20 мг,5 лет,При температуре не выше 30 град.,противоаллергическое средство - H1-гистаминовы...,R06AX29,Нет
4,Аллегра тб п.п.о 180 мг №10,Отпускается без рецепта,drug,924.00,Фексофенадин,тб п.п.о,Санофи Винтроп Индустрия (Франция),NaN,блокатор Н-1 гистаминовых рецепторов,Одна таблетка содержит\n\nАктивное вещество: ф...,...,http://grls.rosminzdrav.ru/Grls_View_v2.aspx?r...,Аллегра,Фексофенадин,"таблетки, покрытые пленочной оболочкой",180 мг,3 года,При температуре не выше 25 град.,противоаллергическое средство - H1-гистаминовы...,R06AX26,Нет


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7877 entries, 0 to 8600
Data columns (total 31 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   name          7877 non-null   object 
 1   prescription  7877 non-null   object 
 2   type          7578 non-null   object 
 3   price         7875 non-null   object 
 4   active_drug   5866 non-null   object 
 5   form          7140 non-null   object 
 6   producer      7877 non-null   object 
 7   country       0 non-null      float64
 8   group         6103 non-null   object 
 9   contains      7765 non-null   object 
 10  description   7817 non-null   object 
 11  action        7151 non-null   object 
 12  use_for       7532 non-null   object 
 13  pregnancy     5621 non-null   object 
 14  how_use       7724 non-null   object 
 15  aware_of      7668 non-null   object 
 16  side_effects  6614 non-null   object 
 17  interaction   6172 non-null   object 
 18  overdose      5872 non-null   obj

In [9]:
df.describe()

,country
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [10]:
df.describe(include=object)

,name,prescription,type,price,active_drug,form,producer,group,contains,description,...,url_grls,TN,INN,DF,dose,date,temp,PG,ATC,VEM
count,7877,7877,7578,7875,5866,7140,7877,6103,7765,7817,...,6362,6264,6264,6267,6267,6267,6267,6264,6264,6264
unique,7162,2,2,4179,1254,196,1092,469,5803,5076,...,4107,3049,1211,370,628,168,235,535,1006,2
top,Пустырника настойка 25 мл,Отпускается без рецепта,drug,1 681.50,Розувастатин,тб,Эвалар ЗАО (Россия),НПВП,"1 таблетка, покрытая пленочной оболочкой, 5 мг...",Прозрачная бесцветная жидкость.,...,http://grls.rosminzdrav.ru/Grls_View_v2.aspx?r...,Гомеопатические монокомпонентные препараты рас...,~,"таблетки, покрытые пленочной оболочкой",~,3 года,При температуре не выше 25 град.,НПВП,~,Нет
freq,9,4403,6361,13,59,1231,168,396,8,43,...,27,27,749,1144,1351,2982,2123,341,536,4176


**Выводы:**

- все признаки представляют собой значения типа object (кроме признака country, информации по которому нет);
- при парсинге данных не была получена информация по признаку country -> возьмем ее из признака producer;
- есть несколько колонок, информацию в которых можно представить в числовом виде (temp, date, price), а также бинаризовать часть колонок (type, prescription, VEM);
- для большинства колонок характерно большое количество уникальных значений -> необходимо рассмотреть возможность уменьшения разнообразия и выделения новых признаков

##  Feature engineering

### VEM, prescription

Введем обозначение 1 для ЖНВЛП и 0 для ЛП, которые в список ЖНВЛП не входят

In [12]:
vem_class = {'Да': 1, 'Нет': 0}
df['VEM'] = df['VEM'].map(vem_class)

Введем обозначение 0 для безрецептурных и 1 для рецептурных препаратов

In [13]:
prescr_class = {'Отпускается по рецепту': 1, 'Отпускается без рецепта': 0}
df['prescription'] = df['prescription'].map(prescr_class)

### country & producer

Заполним колонку country на основе колонки producer

In [14]:
def get_country(data_producer: str) -> Union[str, None]:
    """
    Извлекает информацию о стране производства из признака производитель.

    Parameters
    ----------
    data_producer: str
        Строка с информацией о производителе.

    Returns
    -------
    Union[str, None]
        Страна производства или None, если информация отсутствует.
    """
    if '(' in data_producer:
        return data_producer.split('(')[-1].strip(')')
    return None

In [15]:
df['country'] = df['producer'].apply(get_country)

In [16]:
# посмотрим количество значений
df.country.value_counts()

country
Россия                4182
Германия               617
Индия                  418
Франция                313
Венгрия                196
                      ... 
Исландия                 1
Палестина                1
Египет                   1
Чешская республика       1
Марокко                  1
Name: count, Length: 71, dtype: int64

Добавим флаг, является ли препарат иностранным

In [17]:
df['flg_foreign'] = np.where(df['country'] == 'Россия', 0, 1)

Оставим в колонке producer только наименование производителя

In [18]:
df['producer'] = df['producer'].str.extract(r'^([^()]+)')

### lactation & pregnancy

Введем обозначение 1 для препаратов, которые можно при беременности, и 0 для препаратов, которые противопоказаны при беременности

In [19]:
def allow_pregn(data_aware_of: str, data_pregnancy: str) -> Union[int, None]:
    """
    Извлекает информацию о приеме препарата при беременности.

    Parameters
    ----------
    data_aware_of: str
        Строка с информацией о противопоказаниях.
    data_pregnancy: str
        Строка с информацией о приеме беременными и лактирующими.

    Returns
    -------
    Union[int, None]
        Возвращает 0, если препарат противопоказан при беременности, 
        1 если разрешен и None при отсутствии инормации
    """
    if isinstance(data_aware_of, str) and ('беремен' in data_aware_of.lower()):
        return 0
    elif isinstance(data_pregnancy,
                    str) and (('не рекоменд' in data_pregnancy.lower()) or
                              ('противопоказ' in data_pregnancy.lower())):
        return 0
    elif isinstance(data_pregnancy, str) or isinstance(data_aware_of, str):
        return 1

    return None

In [20]:
df['pregnant'] = df.apply(lambda x: allow_pregn(
    data_aware_of=x['aware_of'], data_pregnancy=x['pregnancy']),
                                    axis=1)

Введем обозначение 1 для препаратов, которые можно при грудном вскармливании, и 0 для препаратов, которые противопоказаны при грудном вскармливании

In [21]:
def allow_lact(data_aware_of: str, data_pregnancy: str) -> Union[int, None]:
    """
    Извлекает информацию о приеме препарата при грудном вскармливании.

    Parameters
    ----------
    data_aware_of: str
        Строка с информацией о противопоказаниях.
    data_pregnancy: str
        Строка с информацией о приеме беременными и лактирующими.

    Returns
    -------
    Union[int, None]
        Возвращает 0, если препарат противопоказан при грудном вскармливании, 
        1 если разрешен и None при отсутствии инормации
    """
    if isinstance(data_aware_of,
                  str) and (('лактац' in data_aware_of.lower()) or
                            ('груд' in data_aware_of.lower())):
        return 0
    elif isinstance(data_pregnancy,
                    str) and (('прекр' in data_pregnancy.lower()) or
                              ('не рекоменд' in data_pregnancy.lower()) or
                              ('противопоказ' in data_pregnancy.lower())):
        return 0
    elif isinstance(data_pregnancy, str) or isinstance(data_aware_of, str):
        return 1

    return None

In [22]:
df['lactation'] = df.apply(lambda x: allow_lact(
    data_aware_of=x['aware_of'], data_pregnancy=x['pregnancy']),
                                     axis=1)

### children

Введем обозначение 1 для препаратов, которые можно детям, и 0 для препаратов, которые противопоказаны детям до 18 лет

In [23]:
def allow_children(data_aware_of: str, data_use: str) -> Union[int, None]:
    """
    Извлекает информацию о приеме препарата детьми.

    Parameters
    ----------
    data_aware_of: str
        Строка с информацией о противопоказаниях (в т.ч. по возрасту).

    Returns
    -------
    Union[int, None]
        Возвращает 0, если препарат противопоказан детям, 
        1 если разрешен и None при отсутствии инормации.
    """
    if isinstance(data_aware_of, str):
        age_pattern = r'(\bдо\s\d+\S*\s*(?:год|лет))'
        ages = re.findall(age_pattern, data_aware_of)
        if ages:
            min_age = min(
                [int(re.search(r'\d+', age).group()) for age in ages])
            if min_age < 18:
                return 1
            else:
                return 0
        else:
            return 1
    elif isinstance(data_use, str):
        if 'дети' in data_use:
            return 1
        else:
            return 0

In [24]:
df['children'] = df.apply(lambda x: allow_children(data_aware_of=x['aware_of'],
                                                   data_use=x['how_use']),
                          axis=1)

### temp & date

Обработаем признак максимальной температуры хранения

In [25]:
def max_temp(data_temp: str, data_storage: str) -> Union[int, None]:
    """
    Функция получения максимальной температуры хранения. 

    Parameters
    ----------
    data_temp: str
        Строка с информацией об условиях хранения.
    data_storage: str
        Строка с информацией об условиях хранения и сроке годности.

    Returns
    -------
    Union[int, None]
        Значение максимальной темп хранения, если найдена, иначе None.
    """
    if isinstance(data_temp, str) and re.findall(r'\d+', data_temp):
        return int(max(re.findall(r'\d+', data_temp)))
    elif isinstance(data_storage, str):
        result = re.findall(
            r'(?:не выше|до|ниже|температуре\s\(*\d+\-)\s*(\d+)',
            data_storage.lower())
        if result:
            return int(result[0])

        return None

In [26]:
df['temp'] = df.apply(
    lambda x: max_temp(data_temp=x['temp'], data_storage=x['storage']), axis=1)

Обработаем признак срока годности в количестве лет

In [27]:
def expiration_date(data_date: str, data_storage: str) -> Union[float, None]:
    """
    Функция получения срок годности в формате float из признака storage

    Parameters
    ----------
    data_date: str
        Строка с информацией о сроке годности.
    data_storage: str
        Строка с информацией об условиях хранения и сроке годности.

    Returns
    -------
    Union[float, None]
        Значение срока годности, если найден, иначе None.
    """
    if isinstance(data_date, str):
        res_year = re.findall(r'^(\d+)\s+(?:год|лет)', data_date)
        res_mon = re.findall(r'^(\d+)\s+(?:мес)', data_date)
        if res_year:
            return int(res_year[0])
        elif res_mon:
            return round(int(res_mon[0]) / 12, 2)
    elif isinstance(data_storage, str):
        res_year = re.findall(r'годности\s+(\d+)\s+(?:год|лет)', data_storage)
        res_mon = re.findall(r'годности\s+(\d+)\s+(?:мес)', data_storage)
        if res_year:
            return int(res_year[0])
        elif res_mon:
            return round(int(res_mon[0]) / 12, 2)

        return None

In [28]:
df['date'] = df.apply(
    lambda x: expiration_date(data_date=x['date'], data_storage=x['storage']),
    axis=1)

### dose & amount

Обработаем признак дозы

In [29]:
def extract_dose(data_dose: str, data_name: str) -> Union[str, None]:
    """
    Функция получения дозировки препарата.

    Parameters
    ----------
    data_dose: str
        Строка с информацией о дозировке.
    data_name: str
        Строка с информацией о наименовании.

    Returns
    -------
    Union[str, None]
        Значение дозировки, если найдена, иначе None.
    """
    if isinstance(data_dose, str) and re.findall(r'\d+', data_dose):
        return data_dose
    else:
        res = re.findall(
            r'\D+[\d+]*\s(\d+(?:[.,]\d+)?\s*(?:мкг|мг\/мл|мг|%|мл|г))',
            data_name)
        if res:
            if ',' in res:
                return res[0].replace(',', '.')
            else:
                return res[0]
        
        return None

In [30]:
df['dose'] = df.apply(
    lambda x: extract_dose(data_dose=x['dose'], data_name=x['name']), axis=1)

Создадим дополнительный признак "количество в упаковке"

In [31]:
def extract_amount(data_name: str) -> Union[int, None]:
    """
    Функция получения информации о количестве в упаковке.

    Parameters
    ----------
    data_name: str
        Строка с информацией о наименовании.

    Returns
    -------
    Union[int, None]
        Значение количества в упаковке, если найдено, иначе None.
    """
    amount = re.findall(r'(\d+)\D*$', data_name)
    if amount:
        return int(amount[0])
    
    return None

In [32]:
df['amount'] = df['name'].apply(extract_amount)

### price

In [33]:
# посмотрим на значения признака price
df['price'].unique()

array(['1\xa0700.50', '388.00', 'от 87.00', ..., '2\xa0618.00',
       '1\xa0041.00', '2\xa0625.50'], dtype=object)

Приведем значения в признаке price к типу float

In [34]:
def change_price(data_price: str) -> float:
    """
    Функция приведения цены к формату float.

    Parameters
    ----------
    data_price: str
        Строка с информацией о цене.

    Returns
    -------
    float
        Значение цены в формате float.
    """
    if isinstance(data_price, str):
        if data_price.startswith('от') and '\xa0' in data_price:
            return float(''.join(data_price.split('\xa0'))[3:])
        elif data_price.startswith('от'):
            return float(data_price[3:])
        elif '\xa0' in data_price:
            return float(''.join(data_price.split('\xa0')))
        elif 'n/a' in data_price:
            return -999.0

        return float(data_price)

In [35]:
df['price'] = df['price'].apply(change_price)

### type

Обработаем признак type - для рецептурных препаратов добавим тип drug

In [36]:
def extract_type(data_type: str, data_prescription: int) -> Union[str, None]:
    """
    Функция получения информации о типе препарата.

    Parameters
    ----------
    data_type: str
        Строка с информацией о типе препарата (ЛП или БАД).
    data_prescription: int
        Строка с информацией о порядке отпуска.

    Returns
    -------
    Union[str, None]
        Значение типа, если найден, иначе None.
    """
    if isinstance(data_type, str):
        return data_type
    elif isinstance(data_prescription, int):
        if data_prescription == 1:
            return 'drug'

    return None

In [37]:
df['type'] = df.apply(lambda x: extract_type(
    data_type=x['type'], data_prescription=x['prescription']),
                             axis=1)

### group PG

Приведем признаки PG и group к одному

In [38]:
def extract_group(data_group: str, data_PG: str) -> Union[str, None]:
    """
    Функция получения информации о группе препарата.

    Parameters
    ----------
    data_group: str
        Строка с информацией о группе.
    data_PG: str
        Строка с информацией о фармако-терапевтической группе.
    data_type: str
        Строка с информацией о типе препарата (ЛП или БАД).

    Returns
    -------
    Union[str, None]
        Значение группы, если найдено, иначе None.
    """
    if isinstance(data_PG, str):
        if data_PG == '~':
            return None
        else:
            return data_PG
    elif isinstance(data_group, str):
        return data_group

    return None

In [39]:
df['group_PG'] = df.apply(
    lambda x: extract_group(data_group=x['group'], data_PG=x['PG']), axis=1)

In [40]:
# создадим список с уникальными значениями признака group
group_lst = list(df['group_PG'].unique())

Заполним значениями группы из action и type

In [41]:
def find_group(data_group: str, data_action: str, data_type: str,
               group_lst: list) -> Union[str, None]:
    """
    Функция получения информации о группе препарата на основе списка групп.

    Parameters
    ----------
    data_group: str
        Строка с информацией о группе.
    data_action: str
        Строка с информацией о фармакологическом действии.
    data_type: str
        Строка с информацией о типе препарата (ЛП или БАД).
    group_lst: list
        Список уникальных значений групп.

    Returns
    -------
    Union[str, None]
        Значение группы, если найдено, иначе None.
    """
    if isinstance(data_group, str):
        return data_group
    elif isinstance(data_type, str):
        if data_type == 'supp':
            return 'БАД'
    elif isinstance(data_action, str):
        for group in group_lst:
            if group:
                if group.lower() in data_action.lower():
                    return group

    return None

In [42]:
df['group_PG'] = df.apply(lambda x: find_group(data_group=x['group_PG'],
                                            data_action=x['action'],
                                            data_type=x['type'],
                                            group_lst=group_lst),
                       axis=1)

### active_drug

Приведем признаки active_drug и INN к одному

In [43]:
def extract_substance(data_active_drug: str,
                      data_INN: str) -> Union[str, None]:
    """
    Функция получения информации о действующем веществе.

    Parameters
    ----------
    data_active_drug: str
        Строка с информацией о действующем веществе.
    data_INN: str
        Строка с информацией о МНН.

    Returns
    -------
    Union[str, None]
        Значение действующего вещества, если найдено, иначе None.
    """
    if isinstance(data_active_drug, str):
        return data_active_drug
    elif isinstance(data_INN, str):
        if data_INN == '~':
            return None
        else:
            return data_INN   

    return None

In [44]:
df['active_drug'] = df.apply(lambda x: extract_substance(
    data_active_drug=x['active_drug'], data_INN=x['INN']),
                             axis=1)

In [45]:
# создадим список с уникальными значениями активных веществ
substance_lst = list(df['active_drug'].unique())

Заполним пропуски значениями из признака contains

In [46]:
def subst_from_contains(data_active_drug: str, data_contains: str,
                        substance_lst: list) -> Union[str, None]:
    """
    Функция получения информации о составе на основе списка действ. веществ.

    Parameters
    ----------
    data_active_drug: str
        Строка с информацией о действующем веществе.
    data_contains: str
        Строка с информацией о фармакологическом действии.
    substance_lst: list
        Список уникальных значений действующих веществ.

    Returns
    -------
    Union[str, None]
        Значение действующего вещества, если найдено, иначе None.
    """
    if isinstance(data_active_drug, str):
        return data_active_drug
    elif isinstance(data_contains, str):
        for substance in substance_lst:
            if substance:
                if '+' in substance:
                    for sub in substance.split('+'):
                        if (sub.lower()
                                in data_contains.lower()) and sub != '':
                            return sub
                else:
                    if substance.lower() in data_contains.lower():
                        return substance

    return None

In [47]:
df['active_drug'] = df.apply(
    lambda x: subst_from_contains(data_active_drug=x['active_drug'],
                                  data_contains=x['contains'],
                                  substance_lst=substance_lst),
    axis=1)

### form

Приведем признаки form и DF к одному

In [48]:
def extract_form(data_form: str, data_DF: str) -> Union[str, None]:
    """
    Функция получения информации о лекарственной форме препарата.

    Parameters
    ----------
    data_form: str
        Строка с информацией о форме выпуска.
    data_DF: str
        Строка с информацией о лекарственно форме.

    Returns
    -------
    Union[str, None]
        Значение лекарственной формы, если найдено, иначе None.
    """
    if isinstance(data_form, str):
        return data_form
    elif isinstance(data_DF, str):
        return data_DF

    return None

In [49]:
df['form'] = df.apply(
    lambda x: extract_form(data_form=x['form'], data_DF=x['DF']), axis=1)

Добавим более общие значения лекарственной формы: 

In [50]:
# создадим словарь с лекарственными формами
forms_dict = {
    'аэрозоль': ['аэр'],
    'гель': ['гель'],
    'гранулы': ['гран'],
    'имплантат': ['имплант'],
    'капли': ['капли'],
    'капсулы': ['капс'],
    'концентрат': ['конц'],
    'крем': ['крем'],
    'линимент': ['лин'],
    'лиофилизат': ['лиоф'],
    'мазь': ['мазь'],
    'масло': ['масло'],
    'настойка': ['настройка', 'н-ка'],
    'палочки': ['палочк'],
    'паста': ['паста'],
    'пена': ['пена'],
    'пластырь': ['пластырь'],
    'пленки': ['пленк'],
    'порошок': ['пор'],
    'раствор': ['р-р', 'раствор', 'жидкость'],
    'пеллеты': ['пеллет'],
    'сироп': ['сир'],
    'спрей': ['спр'],
    'суппозитории': ['супп'],
    'суспензия': ['сусп'],
    'таблетки': ['тб', 'табл'],
    'тампоны': ['тамп'],
    'экстракт': ['экстр'],
    'эмульсия': ['эмульс'],
    'губка': ['губк'],
    'карандаш': ['каранд'],
    'леденцы': ['леден'],
    'пластины': ['пластин'],
    'плитки': ['плит'],
    'резинка жев': ['рез жев'],
    'шампунь': ['шамп'],
    'эликсир': ['эликс'],
    'пилюли': ['пилюл'],
    'сборы': ['сбор'],
    'раст. сырье': [
        'раст.сырье', 'трав', 'корни', 'корень', 'гран. р-пресс.', 'кора',
        'цветк', 'растительное сырье', 'фильтр-пакет', 'ф/п', 'настой',
        'отвар', 'плоды', 'сок'
    ],
    'растворитель': ['растворитель', 'р-ль'],
    'бальзам': ['бальз', 'бал-м'],
    'ТДТС': ['ТДТС'],
    'саше': ['саше'],
    'драже': ['драже', ' др '],
    'пастилки': ['пастилк', 'паст', 'паст жев'],
    'оподельдок':
    'оподельдок'
}

In [51]:
# содадим функцию, которая будет обрабатывать признак form
def check_and_add_form(data_form: str, data_name: str,
                       forms_dict: dict) -> str:
    """
    Функция, которая находит значения ЛФ препарата и приводит их к общему виду.

    Parameters
    ----------
    data_form: str
        Строка с информацией о форме выпуска.
    data_name: str
        Строка с информацией о наименовании.
    forms_dict: dict
        Словарь с значениями лекарственных форм.

    Returns
    -------
    str
        Значение лекарственной формы.
    """
    if isinstance(data_form, str):
        for form in forms_dict:
            for sub_form in forms_dict[form]:
                if sub_form in data_form.lower() or sub_form.strip(
                ) == data_form.lower():
                    return form
        return data_form
    else:
        for form in forms_dict:
            for sub_form in forms_dict[form]:
                if sub_form in data_name.lower():
                    return form

In [52]:
df['form'] = df.apply(lambda x: check_and_add_form(
    data_form=x['form'], data_name=x['name'], forms_dict=forms_dict),
                      axis=1)

### group ATC

Обработаем признак ATC

1. Создадим функцию, которая объединяет файлы CSV в один файл

In [53]:
def ATC_to_file(files_direct: str) -> None:
    """
    Считывает данные из файлов в указанной директории, обрабатывает 
    и объединяет их в один DataFrame, затем сохраняет в файл.

    Parameters:
    -----------
    files_direct : str
        Путь к директории с файлами CSV.

    Returns:
    --------
    None
        Создает обединенный файл CSV.
    """
    data_dir = Path(files_direct)
    df_full = pd.DataFrame()
    for ATC_file in data_dir.glob('*.csv'):
        # добавим группы АТХ (сравнение с таблицей АТХ -> вывод общей группы)
        df_ATC = pd.read_csv(ATC_file,
                             sep=',',
                             skiprows=1,
                             names=['АТХ код', 'Название'])
        # разделим столбец 'АТХ код' на два столбца 'АТХ код' и 'Название'
        df_ATC[['АТХ код', 'Название'
                ]] = df_ATC['АТХ код'].str.split('"', expand=True).drop(2,
                                                                        axis=1)
        df_ATC['АТХ код'] = df_ATC['АТХ код'].str.replace(',', '').str.strip()
        # переименуем столбы
        df_ATC = df_ATC.rename(columns={
            'АТХ код': 'ATC_code',
            'Название': 'ATC_name'
        })
        # добавим в общий датафрейм
        df_full = pd.concat([df_full, df_ATC], ignore_index=True)
    df_full.to_csv("data/ATC.csv", index=False)

2. Применим функцию и загрузим датасет в переменную

In [54]:
# ATC_to_file(files_direct='data/ATC')

In [55]:
df_ATC = pd.read_csv('data/ATC.csv')

3. Создадим функцию для соответсвия кода ATC и название группы, добавим наименование группы group_ATC

In [56]:
def extract_group_ATC(data_ATC: pd.DataFrame, code: str,
                      num: int) -> Union[str, None]:
    """
    Находит соответствие кода АТХ и название группы.

    Parameters:
    -----------
    data_ATC: pd.DataFrame
        Данные с информацией о кодах АТХ и наименовании группы АТХ.
    code: str
        Строка с кодом АТХ.
    num: int
        Величина среза.

    Returns:
    --------
    Union[str, None]
        Возвращает название группы, ели найдено, иначе None.
    """
    if isinstance(code, str):
        for i, j in enumerate(data_ATC['ATC_code']):
            if code[:num] == j:
                return data_ATC['ATC_name'][i]

    return None

In [57]:
df['group_ATC_main'] = df.apply(
    lambda x: extract_group_ATC(data_ATC=df_ATC, code=x['ATC'], num=1), axis=1)

In [58]:
df['group_ATC_sub'] = df.apply(
    lambda x: extract_group_ATC(data_ATC=df_ATC, code=x['ATC'], num=3), axis=1)

### route_kind

Добавим признак "путь введения" route_kind

In [59]:
# создадим словарь
routes_dict = {
    'внутрь': [
        'внутрь', 'после еды', 'перед едой', 'во время еды', 'приема пищи',
        'до еды'
    ],
    'ректально': ['вводить в прямую кишку', 'ректальн'],
    'вагинально': ['вагинальн'],
    'инъекционно':
    ['внутривен', 'в/в', 'внутримыш', 'в/м', 'инъекц', 'подкожно', 'п/к'],
    'ингаляционно/интраназально': ['ингаляционн', 'назальн', 'носовой ход'],
    'наружно': ['наружно', 'на кож'],
    'местно': ['местно', ' глазные ']
}

In [60]:
# напишем функцию, которая будет добавлять признак путь введения
def extract_route(data_use: str, data_rout: str,
                  routes_dict: dict) -> Union[str, None]:
    """
    Находит информацию о пути введения.

    Parameters:
    -----------
    data_use: str
        Строка, содержащая информацию о пути введения.
    data_rout: str
        Строка с информацией о пути введения.
    routes_dict: dict
        Словарь с видами пути введения.

    Returns:
    --------
    Union[str, None]
        Возвращает путь ведения, ели найдено, иначе None.
    """
    if isinstance(data_use, str) and not isinstance(data_rout, str):
        for route in routes_dict:
            for sub_route in routes_dict[route]:
                if sub_route in data_use.lower():
                    return route
        return None


    return data_rout

In [61]:
df['route_kind'] = None
# информация из признака how_use
df['route_kind'] = df.apply(lambda x: extract_route(
    data_use=x['how_use'], data_rout=x['route_kind'], routes_dict=routes_dict),
                            axis=1)
# информация из признака name
df['route_kind'] = df.apply(lambda x: extract_route(
    data_use=x['name'], data_rout=x['route_kind'], routes_dict=routes_dict),
                            axis=1)
# информация из признака action
df['route_kind'] = df.apply(lambda x: extract_route(
    data_use=x['action'], data_rout=x['route_kind'], routes_dict=routes_dict),
                            axis=1)
# информация из признака description
df['route_kind'] = df.apply(lambda x: extract_route(data_use=x['description'],
                                                    data_rout=x['route_kind'],
                                                    routes_dict=routes_dict),
                            axis=1)

### use_for

Приведем признаки use_for, action и description к общему виду

In [62]:
def fill_use_for(data_use: str, data_action: str, data_description: str,
                 data_type: str) -> Union[str, None]:
    """
    Приводит признаки use_for, action и description к общему виду.

    Parameters:
    -----------
    data_use: str
        Строка с информацией о показаниях к применению.
    data_action: str
        Строка с информацией о фармакологическом действии.
    data_description: str
        Строка с информацией об описании.

    Returns:
    --------
    Union[str, None]
        Возвращает показания к применению, ели найдено, иначе None.
    """
    if isinstance(data_use, str):
        return data_use
    elif isinstance(data_type, str):
        if data_type == 'supp':
            if isinstance(data_description, str):
                return data_description
            elif isinstance(data_action, str):
                return data_action
            else:
                return None
        elif data_type == 'drug':
            if isinstance(data_action, str):
                return data_action
            else:
                return None
    else:
        if isinstance(data_description, str):
            return data_description
        elif isinstance(data_action, str):
            return data_action
        else:
            return None

In [63]:
df['use_for'] = df.apply(
    lambda x: fill_use_for(data_use=x['use_for'],
                           data_action=x['action'],
                           data_description=x['description'],
                           data_type=x['type']),
    axis=1)

## Drop features

Уберем лишние колонки

In [64]:
df = df.drop([
    'action', 'description', 'how_use', 'side_effects', 'interaction',
    'storage', 'aware_of', 'overdose', 'precautions', 'url_grls', 'pregnancy',
    'contains', 'PG', 'ATC', 'DF', 'INN', 'TN', 'group', 'country'
],
             axis=1)

# Сохранение датасета в файл csv

In [67]:
df.to_csv('data/data_sum.csv', index=False)